# Creating files for Maps
This notebook combines offers and area demographics into a geojson and csv to be plugged into Kepler.gl

In [1]:
import os
import gzip
import json

import pandas as pd
from tqdm import tqdm

from config import cities
from aggregators import bucket_and_bin

In [2]:
# output
fn_map_att = '../data/output/figs/maps/att_offers.csv'
fn_map_cl = '../data/output/figs/maps/centurylink_offers.csv'
fn_map_verizon = '../data/output/figs/maps/verizon_offers.csv'
fn_map_el = '../data/output/figs/maps/earthlink_offers.csv'

fn_map_acs_att = '../data/output/figs/maps/att_acs.geojson'
fn_map_acs_cl = '../data/output/figs/maps/centurylink_acs.geojson'
fn_map_acs_verizon = '../data/output/figs/maps/verizon_acs.geojson'
fn_map_acs_el = '../data/output/figs/maps/earthlink_acs.geojson'

os.makedirs('../data/output/figs/maps', exist_ok=True)

In [3]:
# inputs
fn_att = '../data/output/speed_price_att.csv.gz'
fn_cl = '../data/output/speed_price_centurylink.csv.gz'
fn_verizon = '../data/output/speed_price_verizon.csv.gz'
fn_el = '../data/output/speed_price_earthlink.csv.gz'

fn_out_shp = '../data/intermediary/census/2019_acs_5_shapes.geojson.gz'

In [4]:
recalculate = True

cols_to_keep = [
#     'address_full',
    'lat', 'lon',
    'speed_down',
    'speed_up',
    'speed_down_bins',
    'package',
]

In [64]:
_df.iloc[0]

address_full                              1009 E TURNEY AVE,PHOENIX,AZ 85014,USA
incorporated_place                                                  Phoenix city
major_city                                                               phoenix
state                                                                         AZ
lat                                                                    33.499563
lon                                                                  -112.059526
availability_status                                          GREEN - exact match
block_group_x                                                        40131086011
collection_datetime                                                   1650137113
provider                                                             CenturyLink
speed_down                                                                  60.0
speed_up                                                                     5.0
speed_unit                  

In [6]:
files = zip([fn_map_att, fn_map_cl, fn_map_verizon, fn_map_el],
            [fn_att, fn_cl, fn_verizon, fn_el],
            [fn_map_acs_att, fn_map_acs_cl, fn_map_acs_verizon, fn_map_acs_el])

for fn_out, fn_in, fn_acs in tqdm(files):
    if not os.path.exists(fn_out) or recalculate:
        
        # make points for the maps
        _df = pd.read_csv(fn_in)
        _df = bucket_and_bin(_df)
        _df[_df.speed_down != 0][cols_to_keep].to_csv(fn_out, index=False)
        try:
            fips = _df['block_group'].apply(lambda x: f"{int(x):012d}"[:2]).unique()
            block_groups = set(_df['block_group'].apply(lambda x: f"{int(x):012d}").unique())
        except:
            fips = _df['block_group_x'].apply(lambda x: f"{int(x):012d}"[:2]).unique()
            block_groups = set(_df['block_group_x'].apply(lambda x: f"{int(x):012d}").unique())
        
        # make chloropleth for acs
        features = []
        for state_fip in fips:
            fn_fip = f'../data/intermediary/maps/acs/tl_2019_{state_fip}_bg.geojson'
            _geojson = json.loads(open(fn_fip).read())
            features.extend(_geojson['features'])
        features = [_ for _ in features if str(_['properties']['GEOID']) in block_groups]
        _geojson['features'] = features
        with open(fn_acs, 'w') as f:
            f.write(json.dumps(_geojson))
            
        features = [_ for _ in features if _['properties']['income_level'] == 'Low']
        _geojson['features'] = features
        fn_acs_ = fn_acs.replace('.geojson', '_low_income.geojson')
        with open(fn_acs_, 'w') as f:
            f.write(json.dumps(_geojson))

4it [00:31,  7.82s/it]
